# Compare Country Trajectories
> Comparing how countries trajectories are similar to Italy, South Korea and Japan

- comments: true
- author: Pratap Vardhan
- hide: true
- categories: [growth, compare, interactive]
- image: images/covid-compare-country-trajectories.png
- permalink: /compare-trajectories/

In [2]:
#hide
import pandas as pd
import altair as alt

In [3]:
#hide_input
source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52]
})

alt.Chart(source).mark_bar().encode(
    x='a',
    y='b'
)

alt.Chart(...)